In [1]:
# Event Hub Namespace Name
NAMESPACE_NAME = "streamingdemoehns01"
# Key Value for the RootManageSharedAccessKey or the key at the namespace level you created and chose to use for this 
KEY_VALUE = "6ObLivEO3O51wB99d7bsCNnaxonwYDgWfBjqIQk/YVU="

# The connection string to your Event Hubs Namespace
connectionString = "Endpoint=sb://{0}.servicebus.chinacloudapi.cn/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey={1};EntityPath=ingestion".format(NAMESPACE_NAME, KEY_VALUE)

# Event Hubs Connection Configuration
ehConf = {
  'eventhubs.connectionString' : connectionString
}

productsSoldStream = spark.readStream.format("eventhubs").options(**ehConf).load()

In [2]:
# When data is streamed into the Event Hub and Spark reads it, the message body will be stored as binary data
# We need to cast the binary data as a string to get the contents of the message
GetMessageData = productsSoldStream.select(productsSoldStream.body.cast("string").alias("body"))

# Here we import some necessary libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Here we establish our schema for the incoming JSON messages
schema = StructType([
  StructField("storeId", IntegerType(), True),
  StructField('producttype', IntegerType(), True),
  StructField("timestamp", StringType(), True),
  StructField("name", StringType(), True),
  StructField("category", StringType(), True),
  StructField("price", DecimalType(18,8), True),
  StructField("quantity", IntegerType(), True)
])

# From the string representation of the message contents we extract the JSON structure using the schema defined above and the from_json() function
FilterForCoughSyrupTransactions = GetMessageData.select(from_json("body", schema=schema).alias("body")).where("body.producttype == 14")

print(FilterForCoughSyrupTransactions)

DataFrame[body: struct<storeId:int,producttype:int,timestamp:string,name:string,category:string,price:decimal(18,8),quantity:int>]

In [3]:
# The connection string to your Event Hubs Namespace
connectionStringAlerting = "Endpoint=sb://{0}.servicebus.chinacloudapi.cn/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey={1};EntityPath=alerting".format(NAMESPACE_NAME, KEY_VALUE)

# Event Hubs Connection Configuration
ehConfAlerting = {
  'eventhubs.connectionString' : connectionStringAlerting
}


FilterForCoughSyrupTransactions.select(FilterForCoughSyrupTransactions.body.cast("string")).writeStream.format("eventhubs").options(**ehConfAlerting).option("checkpointLocation", "/streamingDataDemos/demo/checkpoints05").start()

Out[6]: <pyspark.sql.streaming.StreamingQuery at 0x7f0f608362b0>